# Milvus 简介

## 简介

Milvus 是一个开源的向量数据库，旨在解决向量搜索问题。Milvus 支持向量搜索、向量索引、向量存储、向量搜索服务、向量数据管理、向量数据可视化等功能。

Milvus 支持 Python、Java、C++、Go、Rust、JavaScript 等语言，支持 CPU 和 GPU 硬件加速。

Milvus 支持多种索引类型，包括 Flat、IVF_FLAT、IVF_SQ8、IVF_PQ、HNSW

Milvus 支持多种向量距离计算方式，包括 L2、IP、Cosine。

Milvus 支持多种数据存储方式，包括 Memory、Mmap、S3、MinIO、Local。

Milvus 支持多种数据可视化方式，包括 Milvus Web、Milvus Dashboard、Milvus WebUI、Milvus Grafana。

官网：https://milvus.io/


## 架构

Milvus 2.0 是一款云原生向量数据库，采用​存储与计算分离​的架构设计，所有组件均为无状态组件，极大地增强了系统弹性和灵活性。

![image](assets\image-20240630234444648.png)

整个系统分为四个层次：
 
 接入层（Access Layer）：系统的门面，由一组无状态 proxy 组成。对外提供用户连接的 endpoint，负责验证客户端请求并合并返回结果。
 
 服务层（Coordinator Service）：系统的大脑，负责分配任务给执行节点。协调服务共有四种角色，分别为 root coord、data coord、query coord 和 index coord。
 
 执行节点（Worker Node）：系统的四肢，负责完成协调服务下发的指令和 proxy 发起的数据操作语言（DML）命令。执行节点分为三种角色，分别为 data node、query node 和 index node。
 
 存储服务 （Storage）： 系统的骨骼，负责 Milvus 数据的持久化，分为元数据存储（meta store）、消息存储（log broker）和对象存储（object storage）三个部分。

## 基本概念

### db

数据库

### Collection
包含一组 entity，可以等价于关系型数据库系统（RDBMS）中的表。

### Entity
包含一组 field。field 与实际对象相对应。field 可以是代表对象属性的结构化数据，也可以是代表对象特征的向量。primary key 是用于指代一个 entity 的唯一值。

​**注意：**​ 你可以自定义 primary key，否则 Milvus 将会自动生成 primary key。请注意，目前 Milvus 不支持 primary key 去重，因此有可能在一个 collection 内出现 primary key 相同的 entity。

### Field
Entity 的组成部分。​​Field​​ 可以是结构化数据，例如数字和字符串，也可以是向量。

​**注意：**​

Milvus 2.0 现已支持标量字段过滤。并且，Milvus 2.0 在一个集合中只支持一个主键字段。


### Milvus 与关系型数据库的对应关系如下：

| Milvus向量数据库 | 关系型数据库 |
| ---- | ---- |
| Collection | 表 |
| Entity | 行 |
| Field | 字段 |

### Partition
分区是集合（Collection）的一个分区。Milvus 支持将收集数据划分为物理存储上的多个部分。这个过程称为分区，每个分区可以包含多个段。

### Segment
Milvus 在数据插入时，通过合并数据自动创建的数据文件。一个 collection 可以包含多个 segment。一个 segment 可以包含多个 entity。在搜索中，Milvus 会搜索每个 segment，并返回合并后的结果。

### Sharding
Shard 是指将数据写入操作分散到不同节点上，使 Milvus 能充分利用集群的并行计算能力进行写入。默认情况下，单个 Collection 包含 2 个分片（Shard）。目前 Milvus 采用基于​主键哈希​的分片方式，未来将支持随机分片、自定义分片等更加灵活的分片方式。

​**注意：**​ 分区的意义在于通过划定分区减少数据读取，而分片的意义在于多台机器上并行写入操作。

### 索引
索引基于原始数据构建，可以提高对 collection 数据搜索的速度。Milvus 支持多种​​索引类型​​。为提高查询性能，你可以为每个向量字段指定一种索引类型。目前，一个向量字段仅支持一种索引类型。切换索引类型时，Milvus 自动删除之前的索引。

## 数据类型
```
Primary key field supports:

INT64: numpy.int64
VARCHAR: VARCHAR
Scalar field supports:

BOOL: Boolean (true or false)
INT8: numpy.int8
INT16: numpy.int16
INT32: numpy.int32
INT64: numpy.int64
FLOAT: numpy.float32
DOUBLE: numpy.double
VARCHAR: VARCHAR
JSON: JSON
Array: Array
```

# Milvus 安装

## docker安装 Milvus

```
curl -sfL https://raw.githubusercontent.com/milvus-io/milvus/master/scripts/standalone_embed.sh -o standalone_embed.sh

bash standalone_embed.sh start
```

## docker compose 安装 milvus
### 在线安装
```
wget https://github.com/milvus-io/milvus/releases/download/v2.4.5/milvus-standalone-docker-compose.yml -O docker-compose.yml

sudo docker compose up -d

Creating milvus-etcd  ... done
Creating milvus-minio ... done
Creating milvus-standalone ... done
```

### 离线安装

docker 导出命令
```
docker save etcd:2.3.10 > etcd2_3_10.tar
```

docker 导入命令
```
docker load < etcd2_3_10.tar
docker load < minio2_3_10.tar
docker load < milvus2_3_10.tar
``` 

### 启动 milvus
```
# 启动
docker compose up -d
# 停止
docker compose down
```

## Milvus Python SDK

pip install pymilvus

## Milvus 客户端工具attu

```
Attu是Milvus的官方可视化工具，值得注意的一点是，两者的版本需要匹配

官方文档：https://milvus.io/docs/v2.0.x/attu_install-docker.md

github: https://github.com/milvus-io/attu

访问地址：
http://[你的服务IP]:3000/?#/connect

```

# 常用操作

## 1、数据库链接

In [ ]:
from pymilvus import connections

connections.connect(host="127.0.0.1", port=19530)

connections.connect(host="127.0.0.1", port=19530, db_name="default")

In [ ]:

from pymilvus import MilvusClient
client = MilvusClient(uri="http://127.0.0.1:19530")
client = MilvusClient(uri="http://127.0.0.1:19530",  db_name="default")

## 2、数据库管理

In [2]:
from pymilvus import connections, db

connections.connect(host="127.0.0.1", port=19530)

# 查看数据库列表
db.list_database()

# 添加数据库
# database = db.create_database("knowledge")

# # 删除数据库
# db.drop_database("book")

# # 切换数据库
# db.using_database("knowledge")

# db.list_database()

['default', 'knowledge']

## 3、表管理

### 快速创建表

快速创建表只需要输入表名和向量维度，生成的表只包含两个字段:id(作为主键)和vector(作为矢量字段)，默认情况下启用auto_id和enable_dynamic_field设置。

auto_id：是否自动生成id，默认为true。

enable_dynamic_field：是否可以动态新增字段，默认为false。


In [1]:
from pymilvus import MilvusClient
client = MilvusClient(
    uri="http://127.0.0.1:19530",
    db_name="default"
)

# 创建表
client.create_collection(
    collection_name="knowledge",  # 表名
    dimension=5,   # 向量维度
    auto_id=True,
    enable_dynamic_field=True
)

# 查看表
# client.list_collections()

# 删除表
# client.drop_collection("knowledge")

# 查看表是否存在
# client.has_collection("quick_setup")

# 查看表状态
# client.get_load_state("quick_setup")

# 
# client.get_collection_stats("quick_setup")

# 查看表描述
client.describe_collection(collection_name="knowledge")

['knowledge1', 'quick_setup', 'customized_setup', 'hello_milvus']

### 自定义创建表

In [ ]:
from pymilvus import MilvusClient, DataType
# 3. Create a collection in customized setup mode

# 3.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 3.2. Add fields to schema
schema.add_field(field_name="my_id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="my_vector", datatype=DataType.FLOAT_VECTOR, dim=5)

# 3.3 准备索引parameters
index_params = client.prepare_index_params()

# 3.4 添加索引
index_params.add_index(
    field_name="my_id",
    index_type="STL_SORT"
)

index_params.add_index(
    field_name="my_vector", 
    index_type="IVF_FLAT",
    metric_type="IP",
    params={ "nlist": 128 }
)


### 使用schema创建表

In [ ]:

from pymilvus import FieldSchema,DataType,CollectionSchema,Collection,connections

connections.connect("default", host="127.0.0.1", port="19530")

fields = [
            FieldSchema(name="id", dtype=DataType.INT64, auto_id=True, is_primary=True),
            FieldSchema(name="chunk_id", dtype=DataType.VARCHAR, max_length=256, description="id"),
            FieldSchema(name="file_id", dtype=DataType.VARCHAR, max_length=256, description="文件id"),
            FieldSchema(name="file_valid", dtype=DataType.INT8, default_value=1, description="文件有效"),
            FieldSchema(name="chunk_valid", dtype=DataType.INT8, default_value=0, description="块有效"),
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim = 1536 , description="vector")
        ]
schema = CollectionSchema(fields, "这是一个文档知识库表")

Collection(name="knowledge", schema=schema)

### 加载和释放

In [ ]:
# 7. Load the collection
client.load_collection(
    collection_name="knowledge",
    replica_number=1 # Number of replicas to create on query nodes. Max value is 1 for Milvus Standalone, and no greater than `queryNode.replicas` for Milvus Cluster.
)

res = client.get_load_state(
    collection_name="knowledge"
)

# 8. Release the collection
client.release_collection(
    collection_name="knowledge"
)

res = client.get_load_state(
    collection_name="knowledge"
)

print(res)

### 别名

In [ ]:
# 9.1. Create aliases
client.create_alias(
    collection_name="knowledge",
    alias="bob"
)

# 9.2. List aliases
res = client.list_aliases(
    collection_name="customized_setup_2"
)
print(res)

# 9.3. Describe aliases
res = client.describe_alias(
    alias="bob"
)
print(res)

## 4、分区管理

Milvus 中的分区代表集合的细分。此功能允许将集合的物理存储划分为多个部分，通过将焦点缩小到较小的数据子集而不是整个集合，有助于提高查询性能。创建集合时，至少会自动创建一个名为 _default 的默认分区。您最多可以在一个集合中创建 4,096 个分区。

In [ ]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://gfancy.cn:19530"
)

In [3]:
# 2. Create a collection
if not client.has_collection("quick_setup"): 
    client.create_collection(
        collection_name="quick_setup",
        dimension=5,
    )

In [4]:
# 3. List partitions
client.list_partitions(collection_name="quick_setup")


['_default']

### 创建删除分区

In [11]:
client = MilvusClient(
    uri="http://gfancy.cn:19530",
    db_name="knowledge_zgh"
)

In [13]:
client.create_collection(
    collection_name="quick_setup",
    dimension=5,
)

In [14]:
# 4. Create more partitions
client.create_partition(
    collection_name="quick_setup",
    partition_name="partitionA"
)

client.create_partition(
    collection_name="quick_setup",
    partition_name="partitionB"
)


In [ ]:
# 删除分区
client.drop_partition(
    collection_name="quick_setup",
    partition_name="partitionB"
)

### 查看分区

In [15]:
res = client.list_partitions(collection_name="quick_setup")
print(res)

# 5. Check whether a partition exists
res = client.has_partition(collection_name="quick_setup", partition_name="partitionA")
print(res)

['_default', 'partitionA', 'partitionB']


### 加载释放

In [18]:
# 7. Release a partition
client.release_partitions(
    collection_name="quick_setup",
    partition_names=["partitionA"]
)
client.get_load_state(collection_name="quick_setup",  partition_name="partitionA")

In [20]:
client.load_partitions(
    collection_name="quick_setup",
    partition_names=["partitionA", "partitionB"]
)
client.get_load_state(collection_name="quick_setup",  partition_name="partitionA")

## 3、数据增删改查

- 插入数据
 client.insert

- 更新数据
 client.upsert

- 删除数据
 client.delete

- 查询数据
 client.get



In [ ]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

# 2. Create a collection in quick setup mode
# client.create_collection(
#     collection_name="quick_setup",
#     dimension=5
# )

client.list_collections()

In [ ]:
data=[
    {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682"},
    {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025"},
    {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781"},
    {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "color": "pink_9298"},
    {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "color": "red_4794"},
    {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "color": "yellow_4222"},
    {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "color": "red_9392"},
    {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "color": "grey_8510"},
    {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "color": "white_9381"},
    {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "color": "purple_4976"}
]

res = client.insert(
    collection_name="quick_setup",
    data=data
)

print(res)

In [ ]:
# 6. Upsert data in partitions
data=[
    {"id": 9, "vector": [0.06998888224297328, 0.8582816610326578, -0.9657938677934292, 0.6527905683627726, -0.8668460657158579], "color": "black_3000"},
    {"id": 10, "vector": [0.06998888224297328, 0.8582816610326578, -0.9657938677934292, 0.6527905683627726, -0.8668460657158576], "color": "black_3651"},
    {"id": 11, "vector": [0.6060703043917468, -0.3765080534566074, -0.7710758854987239, 0.36993888322346136, 0.5507513364206531], "color": "grey_2049"},
    {"id": 12, "vector": [-0.9041813104515337, -0.9610546012461163, 0.20033003106083358, 0.11842506351635174, 0.8327356724591011], "color": "blue_6168"},
    {"id": 13, "vector": [0.3202914977909075, -0.7279137773695252, -0.04747830871620273, 0.8266053056909548, 0.8277957187455489], "color": "blue_1672"},
    {"id": 14, "vector": [0.2975811497890859, 0.2946936202691086, 0.5399463833894609, 0.8385334966677529, -0.4450543984655133], "color": "pink_1601"},
    {"id": 15, "vector": [-0.04697464305600074, -0.08509022265734134, 0.9067184632552001, -0.2281912685064822, -0.9747503428652762], "color": "yellow_9925"},
    {"id": 16, "vector": [-0.9363075919673911, -0.8153981031085669, 0.7943039120490902, -0.2093886809842529, 0.0771191335807897], "color": "orange_9872"},
    {"id": 17, "vector": [-0.050451522820639916, 0.18931572752321935, 0.7522886192190488, -0.9071793089474034, 0.6032647330692296], "color": "red_6450"},
    {"id": 18, "vector": [-0.9181544231141592, 0.6700755998126806, -0.014174674636136642, 0.6325780463623432, -0.49662222164032976], "color": "purple_7392"},
    {"id": 19, "vector": [0.11426945899602536, 0.6089190684002581, -0.5842735738352236, 0.057050610092692855, -0.035163433018196244], "color": "pink_4996"}
]

res = client.upsert(
    collection_name="quick_setup",
    data=data
)

print(res)

In [ ]:
# 7. Delete entities
client.delete(
    collection_name="quick_setup",
    filter="id in [4,5,6]"
)

client.delete(
    collection_name="quick_setup",
    ids=[18, 19]
)

In [ ]:
client.get(
    collection_name="quick_setup",
    ids=[0, 1, 2]
)

## 4、构建索引

只有使用索引，向量检索时才快，不然会使用暴力搜索方式。

索引选择可以参考 https://milvus.io/docs/index.md

In [ ]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
    uri="http://localhost:19530"
)

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)

client.create_collection(
    collection_name="customized_setup", 
    schema=schema, 
)

client.list_collections()

In [ ]:
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

client.create_index(
    collection_name="customized_setup",
    index_params=index_params
)

from pymilvus import MilvusClient, DataType

In [ ]:


index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

client.create_index(
    collection_name="knowledge",
    index_params=index_params
)


In [ ]:

client.list_indexes("customized_setup")

client.describe_index(
    collection_name="customized_setup",
    index_name="vector_index"
)

In [ ]:
# 6. Drop index
client.drop_index(
    collection_name="customized_setup",
    index_name="vector_index"
)

## 5、向量检索

### 普通查找
直接在collection全量向量中进行查找Top5

In [ ]:
res = client.search(
    collection_name="test_collection", # Replace with the actual name of your collection
    # Replace with your query vector
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5, # Max. number of search results to return
    output_fields=["color"]
)

# 多query查找
res = client.search(
    collection_name="test_collection", # Replace with the actual name of your collection
    data=[
        [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104],
        [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345]
    ], # Replace with your query vectors
    limit=2, # Max. number of search results to return
    search_params={"metric_type": "IP", "params": {}} # Search parameters
)

# 分区查找
res = client.search(
    collection_name="quick_setup",
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5,
    search_params={"metric_type": "IP", "params": {"level": 1}},
    partition_names=["red"]
)

### filter 查找

In [ ]:
import json

res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    # Replace with your query vector
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5, # Max. number of search results to return    
    filter='color like "red%"'
)
# print(res)
# Convert the output to a formatted JSON string
result = json.dumps(res, indent=4)
print(result)

### Range 查找

In [ ]:
search_params = {    
    "metric_type": "L2",
    "params": {
        "radius": 0.5, # Radius of the search circle
        "range_filter": 1.0 # Range filter to filter out vectors that are not within the search circle
    }
}

res = client.search(
    collection_name="quick_setup", # Replace with the actual name of your collection
    # Replace with your query vector
    data=[[0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]],
    limit=5, # Max. number of search results to return  
    search_params=search_params,
)

### 分组查找

In [ ]:
res = client.search(
    collection_name="group_search", # Collection name
    data=[[0.14529211512077012, 0.9147257273453546, 0.7965055218724449, 0.7009258593102812, 0.5605206522382088]], # Query vector
    search_params={
    "metric_type": "L2",
    "params": {"nprobe": 10},
    }, # Search parameters
    limit=10, # Max. number of search results to return
    group_by_field="doc_id", # Group results by document ID
    output_fields=["doc_id", "passage_id"]
)

# Retrieve the values in the `doc_id` column
doc_ids = [result['entity']['doc_id'] for result in res[0]]


### 参数
```
metric_type：索引算法
params.nprobe
params.level
params.radius
params.range_filter

```

In [ ]:
search_parameters = {
    'metric_type': 'L2',
    'params': {
        'nprobe': 10,
        'level': 1,
        'radius': 1.0,
        'range_filter': 0.8
    }
}